In [2]:
import folium
import branca
import geopandas
from folium.plugins import Search


print(folium.__version__)

0.10.1


In [3]:
states = geopandas.read_file(
    'https://rawcdn.githack.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json',
    driver='GeoJSON'
)

cities = geopandas.read_file(
    'https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_populated_places_simple.geojson',
    driver='GeoJSON'
)

In [4]:
states.describe()

,density
count,52.000000
mean,402.504404
std,1395.100812
min,1.264000
25%,53.440000
50%,100.335000
75%,234.050000
max,10065.000000


In [5]:
states_sorted = states.sort_values(by='density', ascending=False)

states_sorted.head(5).append(states_sorted.tail(5))[['name','density']]

,name,density
8,District of Columbia,10065.000
30,New Jersey,1189.000
51,Puerto Rico,1082.000
39,Rhode Island,1006.000
21,Massachusetts,840.200
31,New Mexico,17.160
34,North Dakota,9.916
26,Montana,6.858
50,Wyoming,5.851
1,Alaska,1.264


In [8]:
us_cities = geopandas.sjoin(cities, states, how='inner', op='within')

pop_ranked_cities = us_cities.sort_values(
    by='pop_max',
    ascending=False
)[
    [
        'nameascii',
        'pop_max',
        'geometry'
    ]
].iloc[:20]

/Users/Gabriel/anaconda3/lib/python3.6/site-packages/geopandas/base.py:104: UserWarning: Cannot generate spatial index: Missing package `rtree`.
  warn("Cannot generate spatial index: Missing package `rtree`.")


AttributeError: 'NoneType' object has no attribute 'intersection'

In [7]:
m = folium.Map(location=[38,-97], zoom_start=4)


style_function = lambda x: {
    'fillColor': colormap(x['properties']['density']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}

stategeo = folium.GeoJson(
    states,
    name='US States',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'density'],
        aliases=['State', 'Density'], 
        localize=True
    )
).add_to(m)

citygeo = folium.GeoJson(
    pop_ranked_cities,
    name='US Cities',
    tooltip=folium.GeoJsonTooltip(
        fields=['nameascii','pop_max'],
        aliases=['','Population Max'],
        localize=True)
).add_to(m)

statesearch = Search(
    layer=stategeo,
    geom_type='Polygon',
    placeholder='Search for a US State',
    collapsed=False,
    search_label='name',
    weight=3
).add_to(m)

citysearch = Search(
    layer=citygeo,
    geom_type='Point',
    placeholder='Search for a US City',
    collapsed=True,
    search_label='nameascii'
).add_to(m)

folium.LayerControl().add_to(m)
colormap.add_to(m)

m

NameError: name 'pop_ranked_cities' is not defined